In [1]:
# Standard library
import time
import math
import random
import datetime
from datetime import datetime, timedelta, timezone

# Third-party libraries
import numpy as np
import ijson

# Matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import FixedLocator
from matplotlib import colors as mcolors
from matplotlib.patches import Polygon as MplPolygon
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Shapely
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

# IPython / Widgets (for interactive use)
from ipywidgets import Checkbox, HBox, VBox, HTML
from IPython.display import display

# For interactive plots (if needed)
%matplotlib widget


In [15]:
def interactive_plot_congestion(found_trajectory, polygons_by_label):
    """
    Plot congestion trajectories with toggleable polygon overlays per label and a color-indicated toggle bar.
    
    Args:
        found_trajectory: List of trajectory dicts
        polygons_by_label: Dict of {label: Polygon or list of Polygons}
    """
    # 🎨 Define consistent colors for congestion levels
    label_colors = {
        'light_sub_20': "#04ff00",  # green
        'heavy_sub_10': "#ff0000",       # red
    }

    # Setup figure
    plt.rc('font', family='serif', size=14)
    fig, ax = plt.subplots(figsize=(10, 3))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.05)

    # Build base scatter from all trajectories
    all_timestamps = []
    all_positions = []
    all_speeds = []

    for traj in found_trajectory:
        all_timestamps.extend(traj["timestamp"])
        all_positions.extend(np.array(traj["x_position"]) / 5280)  # convert to miles
        all_speeds.extend(traj["speed_mph"])

    jet = plt.cm.jet
    colors = [jet(x) for x in np.linspace(1, 0.5, 256)]  # from red to green (inverted)
    green_to_red = LinearSegmentedColormap.from_list('GreenToRed', colors, N=256)

    im = ax.scatter(all_timestamps, all_positions, c=all_speeds, cmap=green_to_red, vmin=0, vmax=80, s=0.1)
    plt.colorbar(im, cax=cax).set_label('Speed (mph)', rotation=90, labelpad=20)
    ax.set_xlabel("Time")
    ax.set_ylabel("Mile marker")

    # Format x-axis (timestamp → time string)
    ticks_loc = ax.get_xticks().tolist()
    ax.xaxis.set_major_locator(FixedLocator(ticks_loc))
    x_datetime = [datetime.fromtimestamp(ts, tz=timezone(timedelta(hours=-6))) for ts in ticks_loc]
    labels = [d.strftime('%H:%M:%S') for d in x_datetime]
    ax.set_xticklabels(labels, rotation=45)

    ax.invert_yaxis()

    # Hover formatter
    def format_hover_info(x, y):
        try:
            dt = datetime.fromtimestamp(x, tz=timezone(timedelta(hours=-6)))
            return f"Time: {dt.strftime('%H:%M')}, Mile: {y:.2f}"
        except:
            return f"Time: {x:.2f}, Mile: {y:.2f}"
    ax.format_coord = format_hover_info

    # 🔲 Setup polygon overlays and toggles
    polygon_patches = {}
    toggles = []

    for label, polygons in polygons_by_label.items():
        color = label_colors.get(label, f"#{random.randint(0, 0xFFFFFF):06x}")
        rgba = mcolors.to_rgba(color, alpha=0.5)
        patches = []

        if isinstance(polygons, list):
            shapes = polygons
        elif hasattr(polygons, "geoms"):
            shapes = list(polygons.geoms)
        else:
            shapes = [polygons]

        for poly in shapes:
            coords = np.array(poly.exterior.coords)
            coords[:, 1] /= 5280  # feet to miles
            patch = MplPolygon(coords, facecolor=rgba, edgecolor='black', linewidth=0.8)
            ax.add_patch(patch)
            patches.append(patch)

        # Create toggle with color indicator
        checkbox = Checkbox(value=True, indent=False, layout={'width': '15px'})
        label_html = HTML(f"<span style='color:{color}; font-weight:bold;'>{label.replace('_', ' ').title()}</span>")
        toggles.append(HBox([checkbox, label_html], layout={'align_items': 'center', 'gap': '4px'}))
        polygon_patches[label] = (patches, checkbox)

    # 🔁 Toggle callback
    def update_display(change=None):
        for label, (patches, checkbox) in polygon_patches.items():
            for patch in patches:
                patch.set_visible(checkbox.value)
        fig.canvas.draw_idle()

    for label, (_, checkbox) in polygon_patches.items():
        checkbox.observe(update_display, names='value')
    update_display()

    # ➕ Display toggles (no legend)
    display(VBox(toggles))

    plt.tight_layout()
    plt.show()


def union_parallelograms(parallelograms):
    print("🔁 Unioning parallelograms...")
    t0 = time.time()
    merged = unary_union(parallelograms)
    t1 = time.time()
    print(f"✅ Union complete in {t1 - t0:.2f} seconds.")
    return merged

from shapely.geometry import Polygon
import numpy as np

def create_tilted_parallelogram(x, t, speed, length=200, time_span=60):
    """
    Create a parallelogram centered at (x, t), tilted according to speed.
    - x in feet, t in seconds
    - speed in mph
    """
    mph_to_ftps = 1.46667
    slope = speed * mph_to_ftps  # ft/s
    
    half_len = length / 2
    half_time = time_span / 2
    
    # Compute corners with tilt
    corners = [
        (t - half_time, x - slope * half_time - half_len),
        (t - half_time, x - slope * half_time + half_len),
        (t + half_time, x + slope * half_time + half_len),
        (t + half_time, x + slope * half_time - half_len),
    ]
    return Polygon(corners)

def build_all_parallelograms(points, speeds, length=200, time_span=60):
    parallelograms = []
    n_points = len(points)
    print(f"Building parallelograms for {n_points} points...")
    for i, ((x, t), speed) in enumerate(zip(points, speeds)):
        if i % 100000 == 0 and i > 0:
            print(f"  Processed {i} points...")
        poly = create_tilted_parallelogram(x, t, speed, length, time_span)
        """if i < 5:  # print only for first 5 parallelograms
            print(f"Parallelogram {i} corners: {list(poly.exterior.coords)}")"""
        parallelograms.append(poly)
    print(f"Done creating {len(parallelograms)} parallelograms.")
    return parallelograms

def filter_polygons_by_area(polygons, min_area=10000):
    """
    Filter polygons, keeping only those with area >= min_area.
    """
    if hasattr(polygons, "geoms"):
        poly_list = list(polygons.geoms)
    else:
        poly_list = [polygons]

    kept_polygons = [poly for poly in poly_list if poly.area >= min_area]

    print(f"Filtered {len(poly_list)} polygons down to {len(kept_polygons)} by area >= {min_area}")
    return kept_polygons



def find_congestion_segments(
    file_path, 
    traj_threshold_seconds=10,
    direction=-1,
    speed_thresh=20,
    filter_speed=True,
    min_distance_feet=250,  # added parameter for minimum distance filter
):
    found_trajectory = []
    date = None
    got_date = False
    i = 0
    with open(file_path, 'r') as input_file:
        parser = ijson.items(input_file, 'item', use_float=True)

        for record in parser:
            if record["direction"] == direction and (record["last_timestamp"] - record["first_timestamp"] > traj_threshold_seconds):
                if not got_date:
                    date = datetime.fromtimestamp(record["first_timestamp"]).date()
                    got_date = True

                x_pos = np.array(record["x_position"])
                timestamp = np.array(record["timestamp"])

                # Calculate speed
                speed = np.diff(x_pos) / np.diff(timestamp) * direction
                speed = np.append(speed[0], speed)  # pad to match length
                speed *= 0.681818  # ft/s to mph

                if filter_speed:
                    i += 1
                    if i % 5000 == 0:
                        print(f"Found {len(found_trajectory)}")
                    mask = (speed > 0) & (speed <= speed_thresh)

                    in_segment = False
                    segment_start = 0

                    for i in range(len(mask)):
                        if mask[i] and not in_segment:
                            segment_start = i
                            in_segment = True
                        elif not mask[i] and in_segment:
                            segment_end = i - 1
                            in_segment = False

                            distance = x_pos[segment_end] - x_pos[segment_start]
                            if distance * direction >= min_distance_feet:
                                new_record = {
                                    "x_position": [x_pos[segment_start], x_pos[segment_end]],
                                    "timestamp": [timestamp[segment_start], timestamp[segment_end]],
                                    "speed_mph": [speed[segment_start], speed[segment_end]],
                                }
                                found_trajectory.append(new_record)

                    if in_segment:
                        segment_end = len(mask) - 1
                        distance = x_pos[segment_end] - x_pos[segment_start]
                        if distance * direction >= min_distance_feet:
                            new_record = {
                                "x_position": [x_pos[segment_start], x_pos[segment_end]],
                                "timestamp": [timestamp[segment_start], timestamp[segment_end]],
                                "speed_mph": [speed[segment_start], speed[segment_end]],
                            }
                            found_trajectory.append(new_record)

                else:
                    # If not filtering, just save the entire trajectory (with all speeds)
                    record["speed_mph"] = speed.tolist()
                    found_trajectory.append(record)

    print(f"{file_path}, {date} → Found {len(found_trajectory)} segments (filter_speed={filter_speed})")
    return found_trajectory, date

def simplify_polygons(polygons, tolerance=1000):
    # If MultiPolygon, simplify each geometry
    if hasattr(polygons, "geoms"):
        simplified_geoms = [poly.simplify(tolerance, preserve_topology=True) for poly in polygons.geoms]
        return MultiPolygon(simplified_geoms)
    elif isinstance(polygons, Polygon):
        return polygons.simplify(tolerance, preserve_topology=True)
    else:
        # If it's a list of polygons
        simplified = [poly.simplify(tolerance, preserve_topology=True) for poly in polygons]
        return simplified



def create_congestion_zones(
    trajs,
    length=250,
    time_span=30,
    min_area=500000,
    simplify_tolerance=5000,
    label=None,
    union=True,
):
    """
    From a list of trajectory segments, extract points/speeds,
    and either return raw parallelograms or a unified simplified polygon.

    Returns:
        - If union=True: simplified unioned polygon
        - If union=False: list of raw parallelograms (no filtering/simplifying)
    """
    print(f"\n🔹 Building congestion polygon for: {label or 'Unnamed'}")
    t0 = time.time()

    # Step 1: Extract points and speeds
    all_points = []
    all_speeds = []
    for traj in trajs:
        if "x_position" in traj:
            all_points.extend(zip(traj["x_position"], traj["timestamp"]))
        else:
            all_points.append((traj["x"], traj["t"]))
        all_speeds.extend(traj["speed_mph"])
    t1 = time.time()
    print(f"✅ Extracted {len(all_points)} points in {t1 - t0:.2f}s.")

    # Step 2: Build parallelograms
    print("🔹 Building parallelograms...")
    parallelograms = build_all_parallelograms(all_points, all_speeds, length=length, time_span=time_span)

    if not union:
        print(f"✅ Returning {len(parallelograms)} raw parallelograms (no union/filter/simplify).")
        return parallelograms

    # Step 3: Union, filter, simplify
    print("🔹 Unioning and simplifying polygons...")
    congestion_polygon = union_parallelograms(parallelograms)
    filtered_by_area = filter_polygons_by_area(congestion_polygon, min_area=min_area)
    simplified = simplify_polygons(filtered_by_area, tolerance=simplify_tolerance)

    t2 = time.time()
    print(f"✅ Polygon processed in {t2 - t1:.2f}s (Total: {t2 - t0:.2f}s).")
    return simplified








In [4]:
first_time = True

In [ ]:
file_paths = [
    "/Users/Ludwig/Documents/GitHub/data_demo/637b023440527bf2daa5932f__post1.json",
    "/Users/Ludwig/Documents/GitHub/data_demo/637c399add50d54aa5af0cf4__post2.json",
    "/Users/Ludwig/Documents/GitHub/data_demo/637d8ea678f0cb97981425dd__post3.json",
    "/Users/Ludwig/Documents/GitHub/data_demo/6380728cdd50d54aa5af0cf5__post5.json",
]

file_paths = [
    "/Users/Ludwig/Documents/GitHub/data_demo/637b023440527bf2daa5932f__post1.json",
]


for path in file_paths:
    
    if first_time:
        trajs_very_light, date_very_light = find_congestion_segments(path, speed_thresh=20)
        trajs_light, _ = find_congestion_segments(path, speed_thresh=10)
        first_time = False


    len_to_cover = 250
    time_to_cover = 5 
    min_area = 5000
    simplify_tolerance = 5000
    union_bool = False

    simplified_polygons_very_light = create_congestion_zones(
        trajs_very_light,
        length=len_to_cover,
        time_span=time_to_cover,
        min_area=min_area,
        simplify_tolerance=simplify_tolerance,
        union=union_bool,
        label="Light < 20mph"
)

    simplified_polygons_light = create_congestion_zones(
        trajs_light,
        length=len_to_cover,
        time_span=time_to_cover,
        min_area=min_area,
        simplify_tolerance=simplify_tolerance,
        union=union_bool,
        label="Heavy < 10mph"
    )




    interactive_plot_congestion(
        trajs_very_light,
        polygons_by_label={
            "light_sub_20": simplified_polygons_very_light,
            "heavy_sub_10": simplified_polygons_light,
        }
    )





🔹 Building congestion polygon for: Light < 20mph
✅ Extracted 135990 points in 0.03s.
🔹 Building parallelograms...
Building parallelograms for 135990 points...
  Processed 100000 points...
Done creating 135990 parallelograms.
✅ Returning 135990 raw parallelograms (no union/filter/simplify).

🔹 Building congestion polygon for: Heavy < 10mph
✅ Extracted 18456 points in 0.00s.
🔹 Building parallelograms...
Building parallelograms for 18456 points...
Done creating 18456 parallelograms.
✅ Returning 18456 raw parallelograms (no union/filter/simplify).
